In [ ]:
# 三种方法

import torch
import torch.nn as nn

torch.save(arg, PATH)

torch.load(PATH)

model.load_state_dict(arg)


- 1. No.1 lazy method, .save

In [ ]:
import torch
import torch.nn as nn


#------- complete model --------
torch.save(model, PATH)


# model class must be defined somewhere
# 上一步保存了模型，所以这里是load路径
model = torch.load(PATH)
model.eavl()



- 2. a recpmmended method, only enough parameters of the training model and to inference

In [ ]:
import torch
import torch.nn as nn

#===== state dict ------
torch.save(model.state_dict(), PATH)


# model must be create again with parameters
# 上一步保存了模型的参数，所以这里要先建模型，然后在导入路径
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()



# code


In [9]:
# model class
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_input_features):
        super().__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = Model(n_input_features=6)  #实例化了Model，得到了初始化的参数，即便没有training


for param in model.parameters():
    print(param)   
    

Parameter containing:
tensor([[ 0.3497, -0.3392,  0.1519, -0.2422,  0.3725,  0.0621]],
       requires_grad=True)
Parameter containing:
tensor([0.2118], requires_grad=True)


- method 1

In [12]:
# training model
FILE = 'model.pth'
torch.save(model, FILE)


# load model, 相当于省去了定义model类的一步
model = torch.load(FILE)   # 加载整个模型
model.eval()    # 推理阶段


# 打印参数，包含权重和偏置
for param in model.parameters():
    print(param)     # 得到权重和偏置



Parameter containing:
tensor([[ 0.3497, -0.3392,  0.1519, -0.2422,  0.3725,  0.0621]],
       requires_grad=True)
Parameter containing:
tensor([0.2118], requires_grad=True)


- method 2

In [15]:
# training model
FILE_ = 'model_.pth'
torch.save(model.state_dict(), FILE_)


# 定义模型，再load model
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE_))  # 加载模型参数
loaded_model.eval()    # 推理阶段


# 打印参数，包含权重和偏置
for param in loaded_model.parameters():
    print(param)     # 得到权重和偏置

Parameter containing:
tensor([[ 0.3497, -0.3392,  0.1519, -0.2422,  0.3725,  0.0621]],
       requires_grad=True)
Parameter containing:
tensor([0.2118], requires_grad=True)


- state_dict长什么样

In [17]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_input_features):
        super().__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = Model(n_input_features=6)  #实例化了Model，得到了初始化的参数，即便没有training

print(model.state_dict())
print(model.state_dict().keys())

OrderedDict([('linear.weight', tensor([[ 0.2538,  0.3261,  0.2457,  0.0239,  0.2082, -0.2203]])), ('linear.bias', tensor([0.2289]))])
odict_keys(['linear.weight', 'linear.bias'])


In [18]:
# 还可以用在优化器上
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
print(optimizer.state_dict())


{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


- 在training过程中可以停在某个地方，并save一个checkpoint

In [19]:
checkpoint = {
    "epoch": 90,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict()
}

torch.save(checkpoint, "checkpoint.pth")


In [20]:
# load checkpoint
loaded_checkpoint = torch.load("checkpoint.pth")

model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(), lr=0)

epoch = loaded_checkpoint["epoch"]
model.load_state_dict(loaded_checkpoint["model_state"])
optimizer.load_state_dict(loaded_checkpoint["optim_state"])

print(model.state_dict())
print(optimizer.state_dict())   # 如果lr是0.01，就load成功了


OrderedDict([('linear.weight', tensor([[ 0.2538,  0.3261,  0.2457,  0.0239,  0.2082, -0.2203]])), ('linear.bias', tensor([0.2289]))])
{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


# - transfer from GPU and CPU

In [ ]:
# GPU to CPU

device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(), PATH)

device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

In [ ]:
# GPU to GPU
device = torch.device('cuda')
model.to(device)
model.save(model.state_dict(), PATH)

model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)   # 把model发送到device


In [ ]:
# GPU to GPU
torch.save(model.state_dict(), PATH)

device = torch.to('cuda')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location='cuda'))
model.to(device)   # 把model发送到device
